In [25]:
import pandas as pd
import json
import httplib
import time
from ratelimiter import RateLimiter
import urllib2

In [64]:
@RateLimiter(max_calls=10, period=6)
def get_full_trips(date_query,dest):
    conn = httplib.HTTPSConnection("public-api.blablacar.com")
    
    headers = {
        'accept': "application/json",
        'key': "f1126089b5f1459f89323c770a7ade4c"
    }
                
    request_url = "/api/v2/trips?fn=Milano&tn=" + dest + "&locale=it_IT&_format=json&cur=EUR&db=" + date_query + "&de=" + date_query + "&limit=1&radius=5"                         
    conn.request("GET", request_url, headers=headers) 
    
    try:
        res = conn.getresponse()
    
        data = res.read().decode("utf-8")
        trips_data = json.loads(data)
        
        res1 = int(trips_data['full_trips_count'])
    
        res2 = {}

        if trips_data['trips']:
            res2['country_code'] = trips_data['trips'][0]['arrival_place']['country_code']
        else:
            res2['country_code'] = 'BOH'
    
        if 'recommended_price' in trips_data:
            res2['recommended_price'] = int(trips_data['recommended_price'])
        else:
            res2['recommended_price'] = -1
    
        if 'duration' in trips_data:
            res2['duration'] = int(trips_data['duration'])
        else:
            res2['duration'] = -1
    
        return res1, res2
    except:
        print 'Something went wrong' + dest + ' ' + date_query
        return -1, {'country_code': 'BOH', 'recommended_price': -1, 'duration':-1}


In [27]:
def dict_two_keys_to_df(d):
    return pd.DataFrame(d.values(), index=pd.MultiIndex.from_tuples(d.keys())).unstack(1)

In [28]:
def dict_one_key_to_df(d):
    return pd.DataFrame.from_dict(d, orient='index')

In [29]:
def get_country_full_trips(dataset):
    df = dataset[['dep_day','arr_city_name']]
    d_daycity = {}
    d_city = {}
    count = 0 # counts requests
    for ind,row in df.iterrows():
        date = row[0]
        arr_city = row[1]
        if (arr_city, date) not in d_daycity:
            count += 1
            if arr_city not in d_city:
                d_daycity[(arr_city, date)],d_city[arr_city] = get_full_trips(date, arr_city)
            else:
                d_daycity[(arr_city, date)], _ = get_full_trips(date, arr_city)
                
    print 'Total requests: ' + str(count)
    return dict_two_keys_to_df(d_daycity), dict_one_key_to_df(d_city)

In [30]:
df = pd.read_csv('./Data/Storico/January2017.csv')

In [31]:
print len(df)

6219


In [32]:
print p1

                0
       2017-01-01
Rovigo          0


In [38]:
df_full, df_cityinfo = get_country_full_trips(df)

Total requests: 3248


In [44]:
df_cityinfo.head()

,duration,recommended_price,country_code
Abano Terme,10800,13,IT
Abbiategrasso,3600,2,IT
Acqui Terme,7200,7,IT
Adria,12600,14,IT
"Aereoporto ""v.catullo""",7200,8,IT


In [45]:
df_full.to_csv('./Data/Storico/Full_Jan.csv')

In [47]:
df_cityinfo.to_csv('./Data/Storico/City_Jan.csv')

In [39]:
df2 = pd.read_csv('./Data/Storico/February2017.csv')
print len(df2)

5970


In [40]:
df_full2, df_cityinfo2 = get_country_full_trips(df2)

Total requests: 2984


In [52]:
df_cityinfo2.tail()

,duration,recommended_price,country_code
casello ferrara nord,10800,14,IT
gessate m2,3000,1,IT
Århus,58800,79,DK
Étrembières,14400,17,FR
Černivci,76200,94,UA


In [50]:
df_full2.to_csv('./Data/Storico/Full_Feb.csv')

In [53]:
df_cityinfo2.to_csv('./Data/Storico/City_Feb.csv')

In [55]:
df3 = pd.read_csv('./Data/Storico/March2017.csv')
print len(df3)

5145


In [56]:
df_full3, df_cityinfo3 = get_country_full_trips(df3)

Total requests: 2753


In [58]:
df_full3.to_csv('./Data/Storico/Full_Mar.csv')

In [59]:
df_cityinfo3.to_csv('./Data/Storico/City_Mar.csv')

In [60]:
df4 = pd.read_csv('./Data/Futuro/March2017.csv')
print len(df4)

319


In [62]:
df_full4, df_cityinfo4 = get_country_full_trips(df4)

Total requests: 145


In [63]:
df_full4.to_csv('./Data/Futuro/Full_Mar.csv')
df_cityinfo4.to_csv('./Data/Futuro/City_Mar.csv')

In [65]:
df5 = pd.read_csv('./Data/Futuro/April2017.csv')
print len(df5)

1175


In [67]:
df_full5, df_cityinfo5 = get_country_full_trips(df5)

Total requests: 828


In [69]:
df_full5.to_csv('./Data/Futuro/Full_Apr.csv')
df_cityinfo5.to_csv('./Data/Futuro/City_Apr.csv')

In [70]:
df6 = pd.read_csv('./Data/Futuro/May2017.csv')
print len(df6)

88


In [71]:
df_full6, df_cityinfo6 = get_country_full_trips(df6)

Total requests: 83


In [72]:
df_full6.to_csv('./Data/Futuro/Full_May.csv')
df_cityinfo6.to_csv('./Data/Futuro/City_May.csv')

In [73]:
df7 = pd.read_csv('./Data/Futuro/June2017.csv')
print len(df7)

49


In [74]:
df_full7, df_cityinfo7 = get_country_full_trips(df7)

Total requests: 49


In [75]:
df_full7.to_csv('./Data/Futuro/Full_Jun.csv')
df_cityinfo7.to_csv('./Data/Futuro/City_Jun.csv')

In [76]:
df8 = pd.read_csv('./Data/Futuro/July2017.csv')
print len(df8)

34


In [77]:
df_full8, df_cityinfo8 = get_country_full_trips(df8)

Total requests: 34


In [78]:
df_full8.to_csv('./Data/Futuro/Full_Jul.csv')
df_cityinfo8.to_csv('./Data/Futuro/City_Jul.csv')